In [3]:
# Loading necessary libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import time

import plotly
import plotly.graph_objs as go
import chart_studio.plotly as py
from chart_studio.plotly import plot, iplot
import plotly.express as px

# Setting pandas to display columns
pd.set_option('display.max_columns', None)

In [4]:
# Loading end of drive file
nfl_small2_end_of_drive = pd.read_csv('nfl_small_end_of_drive.csv',index_col=1,\
    dtype= {'ARI' : 'str','ATL' : 'str', 'BAL' : 'str', 'BUF' : 'str', 'CAR' : 'str',
             'CHI' : 'str', 'CIN' : 'str', 'CLE' : 'str', 'DAL' : 'str', 'DEN' : 'str',
             'DET' : 'str', 'GB' : 'str', 'HOU' : 'str', 'IND' : 'str', 'JAX' : 'str',
             'KC' : 'str', 'LA' : 'str', 'LAC' : 'str', 'MIA' : 'str', 'MIN' : 'str',
             'NE' : 'str', 'NO' : 'str', 'NYG' : 'str','NYJ' : 'str', 'OAK' : 'str',
             'PHI' : 'str', 'PIT' : 'str','SEA' : 'str', 'SF' : 'str', 'TB' : 'str',
             'TEN' : 'str', 'WAS' : 'str'})

# Getting teams to add team matrix for easier filtering
teams = list(nfl_small2_end_of_drive.groupby('posteam').sum().index)

# Creating Home and Away Win Columns
nfl_small2_end_of_drive['home_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              > nfl_small2_end_of_drive['total_away_score'],1,0)

nfl_small2_end_of_drive['away_team_win'] = np.where(nfl_small2_end_of_drive['total_home_score']\
                              < nfl_small2_end_of_drive['total_away_score'],1,0)

In [27]:
# Loading end of game file
nfl_end_of_game = pd.read_csv('nfl_end_of_game.csv',index_col=0)

# Adding numeric win group to allow for correlation
win_dict = {'10+ Wins':2,'Between 6 and 9':1,'5 or Less':0}
nfl_end_of_game['win_group_num'] = nfl_end_of_game['binned'].map(win_dict)

In [7]:
# Team wins overall for sorting
team_wins_overall = []
years = [2015,2016,2017,2018]
year_list = []
team_list = []

for team in teams:
    for year in years:
        home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'H')
                                               ]['home_team_win'])

        away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'A')
                                               ]['away_team_win'])

        # Due to lack of data, normalizing 2018 to 16 week season
        if year == 2018:
            all_wins = np.round((home_wins + away_wins) * 16/14,0)
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
            
        else:
            all_wins = home_wins + away_wins
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)

team_by_wins = pd.DataFrame({'team':team_list,'wins':team_wins_overall,'year':year_list}).sort_values(by=['wins'])

# Creating new agg win team list to resort the dataframe by agg win totals
team_win_list = list(team_by_wins.groupby('team').agg({'wins':'sum'}).sort_values(by=['wins']).index)

# Team wins overall for sorting by agg win totals
team_wins_overall = []
years = [2015,2016,2017,2018]
year_list = []
team_list = []

for team in team_win_list:
    for year in years:
        home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'H')
                                               ]['home_team_win'])

        away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                (nfl_small2_end_of_drive['year'] == year) &\
                                                (nfl_small2_end_of_drive[team] == 'A')
                                               ]['away_team_win'])

        # Due to lack of data, normalizing 2018 to 16 week season
        if year == 2018:
            all_wins = np.round((home_wins + away_wins) * 16/14,0)
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)
            
        else:
            all_wins = home_wins + away_wins
            team_wins_overall.append(round(all_wins,0))
            year_list.append(year)
            team_list.append(team)

team_by_wins = pd.DataFrame({'team':team_list,'wins':team_wins_overall,'year':year_list})

<h1>Weather Effects on NFL Play-By-Play Performance</h1>

A common thought as it pertains to the affects of weather on NFL games is that weather plays an important factor on how teams choose play calling and how they perform given certain weather conditions. We are aiming to understand how weather changes intragame affect how teams perform in the moment and ultimately help determine the outcome.

<h2>Background on American Football</h2> <br>
American Football is played between two teams in which each team is looking to advance a football down a 100 yard field to score a Touchdown (7 pts) or Field Goal (3 pts). Both teams are on the field at the same time, and the team advancing the football is called the <b>Offense (blue)</b> and the team trying to keep the Offense from advancing the football is call the <b>Defense (red).</b>

![Football_field](footballfield.jpg)

<h2>The Game</h2><br>
Each game is made up of 4 15 minute quarters. After the first two quarters, we have Halftime, where the teams will take a short break. Teams will flip a coin to see who starts with the ball at the beginning of the game, and the team that does not start with the ball at the beginning of the game will get the ball at halftime.<br>

<h3>A Series</h3><br>
The team on offense will four plays, or downs, to get 10 yards and reset their down counter. If a team does not get 10 yards on 4 downs, or gives the ball to the other defending team (turnover), then that team turns the ball over. A series ends when a team has turned the ball over or gotten ten or more yards.<br>

<h3>A Drive</h3><br>
A Drive is a collection of sequential series run by the offense until they have either score or turned the ball over (which also happens at halftime).


<h3>Playing conditions</h3>
Teams play in stadiums seating between 60 and 80k people. They range in design, but most fit into two categories; Open Air (or exposed to outside weather) or Enclosed (not exposed to outside weather)<br>

<b>The field can be made up of a couple types of surfaces:</b><br>
-Natural Grass<br>
-Artifical Turf (Used mostly for Enclosed Stadiums, but also in some Open Air Stadiums)<br>

<h2>The NFL</h2><br>
The NFL, or National Football League, is the American league for American Football, and considered the most prevalant in the world. There are 32 teams from across the continental USA split into two conferences call the AFC and NFC.

<img src="NFL_Logo.png" style=width:250px;height:350px>

<h2>The Teams</h2>

<img src="nfl_team_logos.png" style=width:650px;height:1050px>

<h2>The Data</h2>

<h3>Data Attributes:</h3><br>

<b>NFL Play-By-Play:</b><br>
1. gameid
2. tbd


<b>Meteostat Weather:</b><br>

<h2>Overall NFL Stats Background(?)</h2>

In [8]:
# Agg Wins by Team, 4 years
fig = go.Figure(data=[go.Bar(name='Total', x = team_by_wins['team'], y = team_by_wins['wins'])])

layout = go.Layout(
    title = 'Total Wins By Team (2015 - 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'<b>'+'Total Wins by Team'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig

#Add year legend?

Over the course of the 4 years from 2015-2018, there are some clear better than average, middle of the road, and worse than average teams. <b>(add boxes or dividers</b>

In [19]:
# Team wins by year
team_wins_2015 = []
teams = list(team_by_wins['team'].unique())
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2015) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2015) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2015.append(round(all_wins,0))


team_wins_2016 = []
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2016) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2016) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2016.append(round(all_wins,0))

team_wins_2017 = []
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2017) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2017) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = home_wins + away_wins
    
    team_wins_2017.append(round(all_wins,0))
    
team_wins_2018 = []
for team in teams:
    home_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                            (nfl_small2_end_of_drive['year'] == 2018) &\
                                            (nfl_small2_end_of_drive[team] == 'H')
                                           ]['home_team_win'])

    away_wins = sum(nfl_small2_end_of_drive[(nfl_small2_end_of_drive['end_of_game'] == 1) &\
                                                 (nfl_small2_end_of_drive['year'] == 2018) &\
                                                 (nfl_small2_end_of_drive[team] == 'A')
                                           ]['away_team_win'])
    
    all_wins = (home_wins + away_wins)*16/14
    
    team_wins_2018.append(round(all_wins,0))
    
# Chart of Total Wins by Year
fig = go.Figure(data=[go.Bar(name='2015', x = teams, y = team_wins_2015),\
                      go.Bar(name='2016', x = teams, y = team_wins_2016),\
                      go.Bar(name='2017', x = teams, y = team_wins_2017),\
                      go.Bar(name='2018', x = teams, y = team_wins_2018)])

layout = go.Layout(
    title = 'Total Wins By Year (2015 - 2018)',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
    
)

fig.update_layout(
    title={
        'text':'<b>'+'Total Wins by Year'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group',width=1000, height=600)
fig

In [110]:
# Diving into LA Rams
x = list(nfl_end_of_game[(nfl_end_of_game['Team']=='LA')& (nfl_end_of_game['win']==1)].groupby('year')['win'].count().index.astype('str'))
y = nfl_end_of_game[(nfl_end_of_game['Team']=='LA')& (nfl_end_of_game['win']==1)].groupby('year')['win'].count().values

fig = go.Figure()
    
fig.add_trace(go.Bar(name='Total', x = x, y = y))

layout = go.Layout(
    title = 'Total Wins for LA (2015 - 2018)',
    xaxis= dict(title= 'Year',dtick=1,ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'<b>'+'Total Wins for LA (2015 - 2018)'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
)

# fig.add_annotation(text='Sean McVay',xref='x1', yref='y1',x=3,y=10,secondary_y=True)

fig

We also see that while there are consistently good and consistently bad teams, most remain fairly competitive and win/loss totals can shift drastically year to year. For that reason, we need to bucket teams based off yearly performance and cannot compare results for one team year over year.

In [36]:
# Grouping Teams into win buckets by year
labels = ['5 or Less', 'Between 6 and 9','10+ Wins']
team_by_wins['binned'] = pd.cut(team_by_wins['wins'], bins=3, labels=labels)


x2015 = team_by_wins[team_by_wins['year']==2015].groupby('binned').count().index
y2015 = team_by_wins[team_by_wins['year']==2015].groupby('binned').count()['team'].values

x2016 = team_by_wins[team_by_wins['year']==2016].groupby('binned').count().index
y2016 = team_by_wins[team_by_wins['year']==2016].groupby('binned').count()['team'].values

x2017 = team_by_wins[team_by_wins['year']==2017].groupby('binned').count().index
y2017 = team_by_wins[team_by_wins['year']==2017].groupby('binned').count()['team'].values

x2018 = team_by_wins[team_by_wins['year']==2018].groupby('binned').count().index
y2018 = team_by_wins[team_by_wins['year']==2018].groupby('binned').count()['team'].values

fig = go.Figure(data=[go.Bar(name='2015', x = x2015, y = y2015),\
                      go.Bar(name='2016', x = x2016, y = y2016),\
                      go.Bar(name='2017', x = x2017, y = y2017),\
                      go.Bar(name='2018', x = x2018, y = y2018)])

layout = go.Layout(
    title = 'Teams by Win Bucket',
    xaxis= dict(title= 'Team',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Wins',ticklen= 5,zeroline= False)
    
)

fig.update_layout(
    title={
        'text':'<b>'+'Teams by Win Bucket'+'</b>'+'<br>(2015 thru 2018)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        barmode='group',width=1000, height=600)
fig

As expected, year over year, we have very consistent groups across teams from low performers to high. We will use these to genericize actual teams into performance groups. Then we can compare across years more easily, given the limited data we have.

In [29]:
# Finding correlation between some key variables and wins

corr_cols = ['yards','yards_against', 'run_plays', 'run_plays_against', 'pass_plays',\
       'pass_plays_against','yard_diff','to_for', 'to_against', 'to_diff','fg_rate',\
        'fg_rate_against','pass_yds_per_at','pass_run_ratio','run_yds_per_at', 
             'pass_yds_per_at_against', 'run_yds_per_at_against']

corr_df = nfl_end_of_game[corr_cols].apply(lambda x: x.corr(nfl_end_of_game.win)).sort_values(kind="quicksort")

fig = go.Figure(data=[go.Bar(name='Corr', x = corr_df.index, y = corr_df.values)])

layout = go.Layout(
    title = 'Variables by Win Correlation',
    xaxis= dict(title= 'Variable',ticklen= 1,zeroline= False),
    yaxis= dict(title= 'Correlation',ticklen= 5,zeroline= False)
)

fig.update_layout(
    title={
        'text':'<b>'+'Variables by Win Correlation'+'</b>'+'<br>(Based on Individual Game Results)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig

Looking at correlation to wins for certain variables, we can see some clear variables we would love to assess weather impacts on.<br>

<b>Variables we will explore further are:</b><br>
1. <b>pass_run_ratio</b> = Passes / Runs per Game
2. <b>to_diff</b> = Turnover Differential
3. <b>fg_rate</b> = Field Goal Success Rate (doesn't seem highly correlated, but could be more greatly affected by weather)
4. <b>yard_diff</b> = Total Yards minus Total Yards by Opponent

In [34]:
# Create categorical variable and order for binned
nfl_end_of_game['binned'] = pd.Categorical(nfl_end_of_game['binned'], \
                                        categories=['5 or Less','Between 6 and 9','10+ Wins'],\
                                        ordered=True)

fig = go.Figure()

fig.add_trace(go.Bar(name='pass to run',x=nfl_end_of_game.groupby('binned').agg({'pass_run_ratio':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'pass_run_ratio':'mean'})['pass_run_ratio'])
             )

fig.update_layout(
    title={
        'text':'<b>'+'Pass to Run Ratio by Team Wins'+'</b>'+'<br>(Pass Plays / Run Plays)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig

Unsurprisingly, the better teams tend to be more balances offensively and split their plays more between passes and runs. I wouldn't be surprised if this also manifests itself into more wins during non-ideal weather conditions

In [33]:
# TO Margin
fig = go.Figure()

fig.add_trace(go.Bar(name='TO Margin',x=nfl_end_of_game.groupby('binned').agg({'to_diff':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'to_diff':'mean'})['to_diff'])
             )

fig.update_layout(
    title={
        'text':'<b>'+'Turnover Margin by Team Wins'+'</b>'+'<br>(Giveaways Minus Takeways)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig

Also, not surprising, better teams tend to hold onto the ball more as well

In [38]:
nfl_end_of_game.head(3)

,game_id,Team,team_against,score,score_against,yards,yards_against,run_plays,run_plays_against,pass_plays,pass_plays_against,to_for,to_against,fg_at,fg_md,fg_at_against,fg_md_against,pass_yards,pass_yards_against,run_yards,run_yards_against,win,pass_run_ratio,yard_diff,to_diff,fg_rate,fg_rate_against,pass_yds_per_at,run_yds_per_at,pass_yds_per_at_against,run_yds_per_at_against,year,binned,win_group_num
0,2015091000,NE,PIT,28,21,366.0,429.0,23.0,25.0,34.0,42.0,1.0,2.0,0.0,0.0,4.0,4.0,281.0,332.0,82.0,134.0,1.0,1.48,-63.0,1.0,0.5,1.0,8.264706,3.565217,7.904762,5.360000,2015,10+ Wins,2
1,2015091300,CHI,GB,23,31,416.0,336.0,33.0,28.0,36.0,23.0,1.0,0.0,3.0,3.0,1.0,1.0,213.0,189.0,189.0,134.0,0.0,1.09,80.0,-1.0,1.0,1.0,5.916667,5.727273,8.217391,4.785714,2015,5 or Less,0
2,2015091301,LA,SEA,33,29,354.0,338.0,26.0,32.0,29.0,47.0,3.0,1.0,2.0,2.0,3.0,3.0,276.0,219.0,76.0,127.0,1.0,1.12,16.0,-2.0,1.0,1.0,9.517241,2.923077,4.659574,3.968750,2015,5 or Less,0


In [68]:
# Field Goal Rate
fig = go.Figure()

fig.add_trace(go.Bar(name='FG Rate',x=nfl_end_of_game.groupby('binned').agg({'fg_rate':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'fg_rate':'mean'})['fg_rate'])
             )

fig.update_layout(
    title={
        'text':'<b>'+'Field Goal Success Rate'+'</b>'+'<br>(Field Goals Made versus Attempted)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig

Every Win Group performs very well as it relates to making Field Goals. The slight edge for better teams may manifest itself in an extra win here and there.

In [45]:
# Yard Differential
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Yard Diff',x=nfl_end_of_game.groupby('binned').agg({'yard_diff':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'yard_diff':'mean'})['yard_diff']),
                    secondary_y=False
             )

fig.add_trace(go.Scatter(name='Yards Gained',x=nfl_end_of_game.groupby('binned').agg({'yards':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'yards':'mean'})['yards'],yaxis='y2'),
                        secondary_y=True
             )

fig.add_trace(go.Scatter(name='Yards Given Up',x=nfl_end_of_game.groupby('binned').agg({'yards_against':'mean'}).index, 
                     y=nfl_end_of_game.groupby('binned').agg({'yards_against':'mean'})['yards_against']*-1,yaxis='y2'),
                        secondary_y=True
             )

fig.update_layout(
    title={
        'text':'<b>'+'Game Average Yard Differential'+'</b>'+'<br>(Yards Gained Minus Yard Given Up)',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
    
    )

fig

Yard differential is also correlate well with Win Group, but these low differences may not play a huge factor given teams gain on average XXX yards.

<h2>Exploring Effects of Weather</h2><br>
We pulled weather data using an API on Meteostat for locations of all NFL games in our dataset. Locations and game timing were taken into account when joining with play-by-play data. For instance, when the Colts played the Jaguars in London on October 2nd 2016, weather was pulled for London during the time the game was happening. Since weather data is on hour increments, we assumed the same weather lasted an entire hour.<br>

<b>Other Assumptions:</b><br>
1. Games played in a dome were considered "ideal conditions" and assumed weather was zero(?)
2. TBD